# The Cournot Model

Imports and set magics:

In [ ]:
import numpy as np
import warnings 
import scipy.optimize as optimize
import plotly.express as px
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Layout

warnings.filterwarnings("ignore", category=RuntimeWarning)

# Model description

**Cournot Model with n firms** 

Under Cournot competition, the firms determine the quantities, after which the price is determined on the market. The market equilibrium is a Nash equilibrium.

We consider an economy where there are 2 identical firms which produce the same homogenous good. The firms choose simulataneaously the quantity $q_i$ that they wants to produce, respectively $q_1$ and $q_2$. 
It means that the total quantity of the market equals

$$
\begin{aligned}
q = q_1 + q_2 + ... +  q_n
\end{aligned}
$$

and the opponents output is 
$$
\begin{aligned}
q_{-i} = q - q_i
\end{aligned}
$$

The marginal costs of the firm i is $c_i$, with $0 ≤ c_i < a$, where $a$ is a constant and indicates the demand intercept. 

The inverse demand function is defined as : 

$$
\begin{aligned}
p(q)
\end{aligned}
$$







Each firm wants to maximize its profit by choosing its production level by taking the output of the other firm as fixed. The profit function for firm $i$ is given by

$$
\begin{aligned}
\Pi_i (q_{-i}, q_i) &= p(q)*q_i - c_i * q_i\\
&= (p(q_{-i} + q_i) - c_i)*q_i
\end{aligned}
$$

We can rewrite the profit function such it is a function of the quantity. It means that the profit functions for the two firms can be written as

$$
\begin{aligned}
\Pi_i (q_{-i}, q_i) = (a - q_{-i} - q_i - c) * q_i\\
\end{aligned}
$$

The best response functions (BR) maximizes payoff given $q_{-i}$. It means that we take the first order condition of the profit function with respect to $q_i$ and sovle for $q_i$. 

$$
\begin{aligned}
\frac{d \Pi_i}{d q_i} = \frac{d p}{d q_i} * q_i + p - c = 0
\end{aligned}
$$

The BR function is given by

$$
\begin{aligned}
q_i = BR(q_{-i}) = \frac{a - c}{2b} - \frac{1}{2} * q_{-i}

\end{aligned}
$$


## Defining the model

We start by looking at a market with two firms. We define the model and solve a model numerically. 

In [ ]:
# define the demand function and the cost function
def p(q1, q2, a, b):
    return a - b*(q1 + q2)

def cost(q, c):
    return q * c 

# define the profit function for the firms
def firm_profit_1(q1, q2, a, b, c):
    return p(q1, q2, a, b) * q1 - cost(q1, c)

def firm_profit_2(q1, q2, a, b, c):
    return p(q1, q2, a, b) * q2 - cost(q2, c)

# define the best response function for the firm
def BR_1(q2, a, b, c):
    Q1 = optimize.minimize(lambda q0: -firm_profit_1(q0, q2, a, b, c), [0]).x[0]
    return max(Q1, 0)

def BR_2(q1, a, b, c):
    Q2 = optimize.minimize(lambda q0: -firm_profit_2(q1, q0, a, b, c), [0]).x[0]
    return max(Q2, 0)

# define the conditions for the Nash equilibrium
def conditions(q,a, b, c ):
    u = q[0] - BR_1(q[1],a, b, c)
    y = q[1] - BR_2(q[0], a, b, c)
    return [u, y]

# set the initial guess and parameters
initial_guess = [1,1]
a = 11
b = 1
c = 2

# use the fsolve function to find the Nash equilibrium
solver = optimize.fsolve(conditions, initial_guess, args=(a, b, c))
q_star = solver[0]

# print the NE
print(f'The Cournot/Nash Equilibirum is : \n q_1^* = q_2^* = {q_star:.2f} \n')

# Calculate and print the price and profit at the NE
p_star = p(q_star, q_star, a, b)
print(f'The price at the Nash equilibrium is: \n p = {p_star:.2f}')

pi_star = firm_profit_1(q_star, q_star, a, b, c)
print(f'The pi at the Nash equilibrium is: \n pi = {pi_star:.2f}')


In a market with two firms we find that the quilibrium quantities equals 3 for each firm and the price equals 5. The two companies will generally produce the same quantity when they are identical and have exactly the same profit function. The firms will both have a profit equal 3.  

We uses the best response functions for firm 1 and 2 to plot the intersection of the Cournot Equilibrium. As the best response function is given by the quantity of the othe firm, first we find firm 2's best response function and insert it in firm 1's response function to find equilibrium quantities. 

In [ ]:
# define production levels for firm 1
production_level_f1 = np.arange(0, 10, 0.1)

# calculate production levels for the firms using the BR function
production_level_f2 = np.vectorize(BR_2)(production_level_f1, a, b, c)
production_level_f1_2 = np.vectorize(BR_1)(production_level_f2, a, b, c)

# create the scatter plot
fig = px.scatter()

# add lines for firm 1 and 2
fig.add_scatter(x=production_level_f2, y=production_level_f1, mode='lines', name='Firm 1', line=dict(color='red'))
fig.add_scatter(x=production_level_f1, y=production_level_f2, mode='lines', name='Firm 2', line=dict(color='blue'))

# add titles
fig.update_layout(title='Cournot Equilibrium, BR function for firm 1 and firm 2')
fig.update_xaxes(title='Output of firm 1')
fig.update_yaxes(title='Output of firm 2')

fig.show()

From the graph we see that the intersection of the BR functions is at (3, 3), which is the equilibrium quantities. This means that the two firms will produce 3 items each. The BR functions are crossing the other firms output axis equal 4.5. It means that the firm will produce 4.5 items when the other firm is not producing any. This impliesd that the total output of the market is smaller for a monopolist than duopoly market. 

In [ ]:
# create a grid of values for q1 and q2
q1_values = np.linspace(0, 5, 100)
q2_values = np.linspace(0, 5, 100)
q1_grid, q2_grid = np.meshgrid(q1_values, q2_values)

# Compute profit levels for each combination of q1 and q2
profit_1_grid = firm_profit_1(q1_grid, q2_grid, a, b, c)
profit_2_grid = firm_profit_2(q1_grid, q2_grid, a, b, c)

# Create a plot for profit functions
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(q1_grid, q2_grid, profit_2_grid, cmap='coolwarm', alpha=0.8)

# add a dot at the NE point
ax.scatter(q_star, q_star, pi_star, c='r', marker='o')

# set axis labels and title
ax.set_xlabel('q1')
ax.set_ylabel('q2')
ax.set_zlabel('Profit')
ax.set_title('Profit Functions for Firms 2')

plt.show()

The plot above shows the profit function for firm 2. The firms are identical so the plot would be identical for firm 1. The plot shows that when firm 1 is not producing, the profit equals 0. When firm 1 is not producing the firm is producing 4.5 and profit equals around 20. 

The red dot represents the Nash equilibrium point as we found above, where the two firms produce quantities that maximize their profits given their competitors' quantity.

# Different parameter values

The values of the parameters a, b and c has an impact on the results. In the following we look at how different values change the demand and equilibrium. 

In [ ]:
N = 2

# define quantity and price
def cournot(N, a, b, c):
    q = (a - b*N - c*N) / (2*N)  
    p = a - b*q - N*q 
    return p, q

# create plot 
def cournot_model(a, b, c):
    p, q = cournot(N, a, b, c) 
    fig, ax = plt.subplots()
    x = np.linspace(0, (a-b*N)/N, 100)  
    ax.plot(x, a - b*x - N*x, lw=2, label='Demand')  
    ax.plot(q, p, 'ro', label='Equilibrium')  
    ax.set_xlabel('Quantity')
    ax.set_ylabel('Price')
    ax.set_xlim([0, (a-b*N)/N])  
    ax.set_ylim([0, a])  
    ax.legend(loc='best')
    plt.title(f'{N} firms, a={a}, b={b}, c={c}')  
    plt.show()

# create sliders for a, b and c
a_slider = FloatSlider(min=0, max=20, step=0.5, value=10, description='a:', layout=Layout(width='50%'))
b_slider = FloatSlider(min=0, max=5, step=0.1, value=1, description='b:', layout=Layout(width='50%'))
c_slider = FloatSlider(min=0, max=5, step=0.1, value=1, description='c:', layout=Layout(width='50%'))

# use the sliders to interact with the cournot_model function
interact(cournot_model, a=a_slider, b=b_slider, c=c_slider)


The parameter a is the demand in the market when the price is zero. So a high value a implies a high demand for the good in the market. 

The parameter b is the slope of the demand. For b going toward infinity, the dependence on the other firm's price on one's own production increases. A high value of b therefore means a the demand curve becomes steeper, and a larger decrease in the market price occurs for each unit increase in the quantity produced. This has the effect of reducing the quantity of the good that each firm produces in equilibrium. 

The parameter c is the cost of production for the firm. It is the cost for the firm to produce one more unit. If c increases, the marginal cost of production for the firm increases, which makes it more expensive for firms to produce each unit of output. 

# Different number of firms in the market

We start by looking at the change when there are three firms in the market. 

In [ ]:
# define the demand function and cost function
def p(q1, q2, q3, a, b):
    return a - b*(q1 + q2 + q3)

def cost(q, c):
    return q * c 

# define the profit function for firm 1, firm 2 and firm 3
def firm_profit_1(q1, q2, q3, a, b, c):
    return p(q1, q2, q3, a, b) * q1 - cost(q1, c)

def firm_profit_2(q1, q2, q3, a, b, c):
    return p(q1, q2, q3, a, b) * q2 - cost(q2, c)

def firm_profit_3(q1, q2, q3, a, b, c):
    return p(q1, q2, q3, a, b) * q3 - cost(q3, c)

# Define the best response function for firm 1, firm 2 and firm 3
def BR_1(q2, q3, a, b, c):
    Q1 = optimize.minimize(lambda q0: -firm_profit_1(q0, q2, q3, a, b, c), [0]).x[0]
    return Q1

def BR_2(q1, q3, a, b, c):
    Q2 = optimize.minimize(lambda q0: -firm_profit_2(q1, q0, q3, a, b, c), [0]).x[0]
    return Q2

def BR_3(q1, q2, a, b, c):
    Q3 = optimize.minimize(lambda q0: -firm_profit_3(q1, q2, q0, a, b, c), [0]).x[0]
    return Q3

# define the conditions for NE
def conditions(q,a, b, c ):
    u = q[0] - BR_1(q[1], q[2], a, b, c) # condition for firm 1
    y = q[1] - BR_2(q[0], q[2], a, b, c) # condition for firm 2
    z = q[2] - BR_3(q[0], q[1], a, b, c) # condition for firm 3
    return [u, y, z]

# set the initial guess
initial_guess = [1,1,1]

# solve for NE
solver = optimize.fsolve(conditions,initial_guess, args = (a, b, c))
q_star = solver
print(f'The Cournot/Nash Equilibirum is : \n q_1^* = {q_star[0]:.2f}, q_2^* = {q_star[1]:.2f}, q_3^* = {q_star[2]:.2f}\n')

# Calculate and print the price and profit at the NE
p_star = p(q_star[0], q_star[1], q_star[2], a, b)
print(f'The price at the Nash equilibrium is: \n p = {p_star:.2f}')

pi_star_i = firm_profit_1(q_star[0], q_star[1], q_star[2], a, b, c)
print(f'As the firms are identical, the profit of firm i at the Nash equilibrium is: \n pi_i = {pi_star_i:.2f}')


Having three firms in the market we get a lower equilibirum price where the price decreases from 5.0 to 4.25. The quantity decreases for each firm but the overall produced quantity for the market will increase which means that the total quantity of the market equals 6.75 compared to 6.0 when having two firms in the market. 

If the number of firms would increase further towards infinity, the equilibrium price will go towards the cost

$$
\begin{aligned}
p^* \rightarrow c
\end{aligned}
$$

and the profit will go towards 0

$$
\begin{aligned}
\pi \rightarrow 0
\end{aligned}
$$

This corresponds to the results that we would get under perfect competition. The price will be pushed down to the marginal cost, the quantity will increase, the profit will go towards 0 and consumer surplus will increase. Exactly as under perfect competition. 

# Conclusion

We have looked a Cournot model with different parameter values and different number of firms in the market.

When having two firms in the market we got a equilibrium price equal 5.00 and equilbrium quantity equal 3.00. When increasing tbe number of firms in the market the equilibrium price decreased but the total quantity of the market increased. 

By increasing the number of firms in the market further, the price will go towards the marginal cost, the quantity will increase and the profit will go towards 0. 